# Corporación Favorita Grocery Sales Forecasting

https://www.kaggle.com/c/favorita-grocery-sales-forecasting/overview

This is an extensive Exploratory Data Analysis for the Corporación Favorita Grocery Sales Forecasting competition.

The aim of this challenge is to forecast more accurate product sales for the Ecuadorian supermarket chain Corporación Favorita.

The data comes in the shape of multiple files. First, the training data (../input/train.csv) essentially contains the sales by date, store, and item. The test data (../input/test.csv) contains the same features without the sales information, which we are tasked to predict. The train vs test split is based on the date. In addition, some test items are not included in the train data.

Furthermore, there are 5 additional data files that provide the following information:

stores.csv: Details about the stores, such as location and type.

items.csv: Item metadata, such as class and whether they are perishable. Note, that perishable items have a higher scoring weight than others.

transactions.csv: Count of sales transactions for the training data

oil.csv: Daily oil price. This is relevant, because “Ecuador is an oil-dependent country and its economical health is highly vulnerable to shocks in oil prices.” (source)

holidays_events.csv: Holidays in Ecuador. Some holidays can be transferred to another day (possibly from weekend to weekday).

In [1]:
import sys
print(sys.version)
import platform
print(platform.python_version())

3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
3.7.6


In [2]:
from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb

## load data

In [4]:
df_train = pd.read_csv(
    'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    "items.csv",
).set_index("item_nbr")

stores = pd.read_csv(
    "stores.csv",
).set_index("store_nbr")

In [5]:
df_train.head(5)

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2016-01-01,25,105574,2.564949,False
1,2016-01-01,25,105575,2.302585,False
2,2016-01-01,25,105857,1.386294,False
3,2016-01-01,25,108634,1.386294,False
4,2016-01-01,25,108701,1.098612,True


In [6]:
df_test.head(5)

id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False

In [7]:
items.head(5)

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1


In [9]:
stores.head(5)

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


## encoder categorical feature

In [10]:
le = LabelEncoder()
items['family'] = le.fit_transform(items['family'].values)

stores['city'] = le.fit_transform(stores['city'].values)
stores['state'] = le.fit_transform(stores['state'].values)
stores['type'] = le.fit_transform(stores['type'].values)

In [11]:
items.head(5)

,family,class,perishable
item_nbr,,,
96995,12,1093,0
99197,12,1067,0
103501,7,3008,0
103520,12,1028,0
103665,5,2712,1


In [12]:
stores.head(5)

,city,state,type,cluster
store_nbr,,,,
1,18,12,3,13
2,18,12,3,13
3,18,12,3,8
4,18,12,3,9
5,21,14,3,4


## prepare data

In [13]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

C:\Users\edward chen\Anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


In [15]:
df_2017.head(5)

,date,store_nbr,item_nbr,unit_sales,onpromotion
35229871,2017-01-01,25,99197,0.693147,False
35229872,2017-01-01,25,103665,2.079442,False
35229873,2017-01-01,25,105574,0.693147,False
35229874,2017-01-01,25,105857,1.609438,False
35229875,2017-01-01,25,106716,1.098612,False


In [14]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.head(5)

onpromotion                                              \
date                2017-01-01 2017-01-02 2017-01-03 2017-01-04 2017-01-05   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   
          105574         False      False       True      False      False   

                                                                           \
date               2017-01-06 2017-01-07 2017-01-08 2017-01-09 2017-01-10   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574         True      False      False      False      False   

                    ...                                              \
date                ... 2017-08-06 2017-08-07 2017-08-08 2017-08-09   
store_nbr item_nbr  ...                                               
1         96995     ...      False      False      False      False   
          99197     ...      False      False      False      False   
          103520    ...      False      False      False      False   
          103665    ...      False      False      False      False   
          105574    ...      False      False      False      False   

                                                                           \
date               2017-08-10 2017-08-11 2017-08-12 2017-08-13 2017-08-14   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   
          105574        False      False      False      False      False   

                               
date               2017-08-15  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103520        False  
          103665        False  
          105574        False  

[5 rows x 227 columns]

In [16]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
print(promo_2017_train.columns)

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
               '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
               '2017-08-14', '2017-08-15'],
              dtype='datetime64[ns]', name='date', length=227, freq=None)


In [17]:
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

In [18]:
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [56]:
promo_2017.head(5)

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   

date                2017-01-09  2017-01-10  ...  2017-08-22  2017-08-23  \
store_nbr item_nbr                          ...                           
1         96995          False       False  ...       False       False   
          99197          False       False  ...       False       False   
          103520         False       False  ...       False       False   
          103665         False       False  ...       False       False   
          105574         False       False  ...       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          103520         False       False       False       False  
          103665         False       False       False       False  
          105574         False       False       False       False  

[5 rows x 243 columns]

In [19]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)
df_2017.head(5)

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   

date                2017-01-09  2017-01-10  ...  2017-08-06  2017-08-07  \
store_nbr item_nbr                          ...                           
1         96995       0.000000    0.000000  ...    1.098612    1.098612   
          99197       0.000000    0.693147  ...    0.000000    1.098612   
          103520      0.693147    0.693147  ...    0.000000    0.000000   
          103665      0.000000    2.079442  ...    0.693147    1.098612   
          105574      1.386294    2.302585  ...    0.000000    1.791759   

date                2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.693147    0.000000   
          99197       0.000000    1.098612    0.000000    0.000000   
          103520      1.386294    0.000000    1.386294    0.693147   
          103665      0.000000    2.079442    2.302585    1.098612   
          105574      2.079442    1.945910    2.397895    1.791759   

date                2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                  
1         96995       0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000  
          103520      0.693147    0.693147    0.000000    0.000000  
          103665      0.000000    0.000000    0.693147    0.693147  
          105574      1.791759    0.000000    1.386294    1.609438  

[5 rows x 227 columns]

In [55]:
print(df_2017.columns)

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
               '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
               '2017-08-14', '2017-08-15'],
              dtype='datetime64[ns]', name='date', length=227, freq=None)


In [22]:
items = items.reindex(df_2017.index.get_level_values(1))
stores = stores.reindex(df_2017.index.get_level_values(0))

In [21]:
items.head(5)

,family,class,perishable
item_nbr,,,
96995,12,1093,0
99197,12,1067,0
103520,12,1028,0
103665,5,2712,1
105574,12,1045,0


In [23]:
stores.head(5)

,city,state,type,cluster
store_nbr,,,,
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13


In [24]:
df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()
df_2017_item.head(5)

date,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,...,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15
item_nbr,,,,,,,,,,,,,,,,,,,,,
96995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.662960,7.454720,2.484907,5.950643,5.545177,8.841014,7.742402,4.969813,7.167038,7.742402
99197,0.693147,17.422746,16.604036,20.569303,16.203025,16.278613,14.775909,17.317386,14.986630,15.833927,...,3.178054,4.969813,3.178054,4.969813,2.079442,2.197225,1.386294,2.079442,0.000000,0.000000
103501,0.000000,55.868320,54.627085,42.810313,39.555298,35.717635,47.208504,47.542538,40.189274,39.200893,...,38.578235,33.531460,35.296421,35.584104,26.270815,32.776619,34.416498,36.546914,34.773173,35.512841
103520,0.000000,38.875486,35.822995,34.979211,42.252967,51.397412,49.505990,33.846832,33.336007,31.741073,...,35.630624,32.567752,47.213872,41.198030,43.569852,48.697330,47.015385,39.070042,33.798042,40.030669
103665,2.079442,56.225402,40.233610,46.138063,38.100507,49.690810,54.725492,54.286513,39.602739,35.899957,...,50.919628,41.262812,34.711732,34.095546,48.162787,50.980653,39.807856,39.016553,34.262348,35.741351


In [25]:
promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()
promo_2017_item.head(5)

date,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,...,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
item_nbr,,,,,,,,,,,,,,,,,,,,,
96995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
99197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
103520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
103665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,1.0,0.0


In [26]:
df_2017_store_class = df_2017.reset_index()
df_2017_store_class['class'] = items['class'].values
df_2017_store_class_index = df_2017_store_class[['class', 'store_nbr']]
df_2017_store_class = df_2017_store_class.groupby(['class', 'store_nbr'])[df_2017.columns].sum()
df_2017_store_class.head(5)

date             2017-01-01  2017-01-02  2017-01-03  2017-01-04  2017-01-05  \
class store_nbr                                                               
1002  1                 0.0    6.291569   11.901285    9.939627   12.817576   
      2                 0.0   27.836761   21.942946   23.265525   20.405583   
      3                 0.0   42.484074   29.286804   35.991684   29.124900   
      4                 0.0   28.353452   21.278199   22.805993   20.207757   
      5                 0.0   19.157935   15.744315   14.909440   12.177673   

date             2017-01-06  2017-01-07  2017-01-08  2017-01-09  2017-01-10  \
class store_nbr                                                               
1002  1           10.961278   13.708549    7.454720   18.598319   12.322254   
      2           23.207544   32.629193   33.057327   18.878450   24.403008   
      3           31.628492   36.412191   32.819483   27.527092   26.893368   
      4           19.822911   24.720218   31.524160   21.634874   17.617490   
      5           12.765460   12.306750   14.427014   15.168951   12.465355   

date             ...  2017-08-06  2017-08-07  2017-08-08  2017-08-09  \
class store_nbr  ...                                                   
1002  1          ...    6.068426   11.966952   13.458607   11.431281   
      2          ...   28.459131   25.885741   22.351737   21.326989   
      3          ...   38.142670   27.082659   30.859413   28.748667   
      4          ...   26.304582   10.961278   17.278515   13.223041   
      5          ...   12.647677   13.969433   11.901285   15.333254   

date             2017-08-10  2017-08-11  2017-08-12  2017-08-13  2017-08-14  \
class store_nbr                                                               
1002  1           13.367622    5.545177   13.628506    5.375278   18.639141   
      2           16.401095   26.828830   27.567914   25.059788   21.576931   
      3           30.232542   36.638416   31.272846   32.256757   33.608285   
      4           17.735273   19.822911   16.806560   23.187741   16.267563   
      5           13.851650   10.332669   14.416963   15.538049   11.192390   

date             2017-08-15  
class store_nbr              
1002  1           10.450452  
      2           24.121024  
      3           24.929109  
      4           16.267563  
      5           11.443704  

[5 rows x 227 columns]

In [27]:
df_2017_promo_store_class = promo_2017.reset_index()
df_2017_promo_store_class['class'] = items['class'].values
df_2017_promo_store_class_index = df_2017_promo_store_class[['class', 'store_nbr']]
df_2017_promo_store_class = df_2017_promo_store_class.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()
df_2017_promo_store_class.head(5)

date             2017-01-01  2017-01-02  2017-01-03  2017-01-04  2017-01-05  \
class store_nbr                                                               
1002  1                 0.0         0.0         0.0         0.0         0.0   
      2                 0.0         0.0         0.0         0.0         0.0   
      3                 0.0         0.0         0.0         0.0         0.0   
      4                 0.0         0.0         0.0         0.0         0.0   
      5                 0.0         0.0         0.0         0.0         0.0   

date             2017-01-06  2017-01-07  2017-01-08  2017-01-09  2017-01-10  \
class store_nbr                                                               
1002  1                 0.0         0.0         0.0         0.0         0.0   
      2                 0.0         0.0         0.0         0.0         0.0   
      3                 0.0         0.0         0.0         0.0         0.0   
      4                 0.0         0.0         0.0         0.0         0.0   
      5                 0.0         0.0         0.0         0.0         1.0   

date             ...  2017-08-22  2017-08-23  2017-08-24  2017-08-25  \
class store_nbr  ...                                                   
1002  1          ...         1.0         1.0         1.0         1.0   
      2          ...         1.0         1.0         1.0         1.0   
      3          ...         1.0         1.0         1.0         1.0   
      4          ...         1.0         1.0         1.0         1.0   
      5          ...         0.0         1.0         1.0         0.0   

date             2017-08-26  2017-08-27  2017-08-28  2017-08-29  2017-08-30  \
class store_nbr                                                               
1002  1                 1.0         1.0         1.0         1.0         1.0   
      2                 1.0         1.0         1.0         1.0         1.0   
      3                 1.0         1.0         1.0         1.0         1.0   
      4                 1.0         1.0         1.0         1.0         1.0   
      5                 1.0         1.0         1.0         1.0         1.0   

date             2017-08-31  
class store_nbr              
1002  1                 1.0  
      2                 1.0  
      3                 1.0  
      4                 2.0  
      5                 0.0  

[5 rows x 243 columns]

In [28]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [29]:
def prepare_dataset(df, promo_df, t2017, is_train=True, name_prefix=None):
    X = {
        "promo_14_2017": get_timespan(promo_df, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_df, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_df, t2017, 140, 140).sum(axis=1).values,
        "promo_3_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 3).sum(axis=1).values,
        "promo_7_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 7).sum(axis=1).values,
        "promo_14_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 14).sum(axis=1).values,
    }

    for i in [3, 7, 14, 30, 60, 140]:
        tmp1 = get_timespan(df, t2017, i, i)
        tmp2 = (get_timespan(promo_df, t2017, i, i) > 0) * 1

        X['has_promo_mean_%s' % i] = (tmp1 * tmp2.replace(0, np.nan)).mean(axis=1).values
        X['has_promo_mean_%s_decay' % i] = (tmp1 * tmp2.replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values

        X['no_promo_mean_%s' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan)).mean(axis=1).values
        X['no_promo_mean_%s_decay' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values

    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s_decay' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s' % i] = tmp.mean(axis=1).values
        X['median_%s' % i] = tmp.median(axis=1).values
        X['min_%s' % i] = tmp.min(axis=1).values
        X['max_%s' % i] = tmp.max(axis=1).values
        X['std_%s' % i] = tmp.std(axis=1).values

    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017 + timedelta(days=-7), i, i)
        X['diff_%s_mean_2' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s_decay_2' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s_2' % i] = tmp.mean(axis=1).values
        X['median_%s_2' % i] = tmp.median(axis=1).values
        X['min_%s_2' % i] = tmp.min(axis=1).values
        X['max_%s_2' % i] = tmp.max(axis=1).values
        X['std_%s_2' % i] = tmp.std(axis=1).values

    for i in [7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)
        X['has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

        tmp = get_timespan(promo_df, t2017, i, i)
        X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_promo_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

    tmp = get_timespan(promo_df, t2017 + timedelta(days=16), 15, 15)
    X['has_promo_days_in_after_15_days'] = (tmp > 0).sum(axis=1).values
    X['last_has_promo_day_in_after_15_days'] = i - ((tmp > 0) * np.arange(15)).max(axis=1).values
    X['first_has_promo_day_in_after_15_days'] = ((tmp > 0) * np.arange(15, 0, -1)).max(axis=1).values

    for i in range(1, 16):
        X['day_%s_2017' % i] = get_timespan(df, t2017, i, 1).values.ravel()

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, t2017, 140-i, 20, freq='7D').mean(axis=1).values

    for i in range(-16, 16):
        X["promo_{}".format(i)] = promo_df[t2017 + timedelta(days=i)].values.astype(np.uint8)

    X = pd.DataFrame(X)

    if is_train:
        y = df[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    return X

In [47]:
print("Preparing dataset...")
t2017 = date(2017, 6, 14)
num_days = 6
X_l, y_l = [], []
for i in range(num_days):
    print("="*40)
    print("\n num_days: {}".format(i))
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(df_2017, promo_2017, t2017 + delta)
    
    print("X_tmp:\n {}".format(X_tmp.head(5)))
    print("y_tmp:\n {}".format(X_tmp.head(5)))
    X_tmp2 = prepare_dataset(df_2017_item, promo_2017_item, t2017 + delta, is_train=False, name_prefix='item')
    X_tmp2.index = df_2017_item.index
    print("X_tmp2 before reset_index:\n {}".format(X_tmp2.head(5)))
    print(df_2017.index.get_level_values(1))
    X_tmp2 = X_tmp2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)
    print("X_tmp2 after reset_index:\n {}".format(X_tmp2.head(5)))
    
    X_tmp3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, t2017 + delta, is_train=False, name_prefix='store_class')
    X_tmp3.index = df_2017_store_class.index
    print("X_tmp3 before reset_index:\n {}".format(X_tmp3.head(5)))
    #X_tmp3 = X_tmp3.reindex(df_2017_store_class_index).reset_index(drop=True)
    print(df_2017_store_class.index.dtype)
    X_tmp3 = X_tmp3.reindex(df_2017_store_class.index).reset_index(drop=True)
    print("X_tmp3 after reset_index:\n {}".format(X_tmp3.head(5)))
    
    X_tmp = pd.concat([X_tmp, X_tmp2, X_tmp3, items.reset_index(), stores.reset_index()], axis=1)
    X_l.append(X_tmp)
    y_l.append(y_tmp)

    del X_tmp2
    gc.collect()

Preparing dataset...

 num_days: 0
X_tmp:
    promo_14_2017  promo_60_2017  promo_140_2017  promo_3_2017_aft  \
0              0              0               0                 0   
1              0              0               0                 0   
2              0              0               0                 0   
3              0              0               0                 0   
4              0              8              34                 0   

   promo_7_2017_aft  promo_14_2017_aft  has_promo_mean_3  \
0                 0                  0               NaN   
1                 0                  0               NaN   
2                 0                  0               NaN   
3                 0                  0               NaN   
4                 0                  0               NaN   

   has_promo_mean_3_decay  no_promo_mean_3  no_promo_mean_3_decay  ...  \
0                     0.0         0.000000               0.000000  ...   
1                     0.0        

X_tmp3 before reset_index:
                  store_class_promo_14_2017  store_class_promo_60_2017  \
class store_nbr                                                         
1002  1                               13.0                       22.0   
      2                               17.0                       33.0   
      3                               19.0                       34.0   
      4                               15.0                       21.0   
      5                                3.0                        6.0   

                 store_class_promo_140_2017  store_class_promo_3_2017_aft  \
class store_nbr                                                             
1002  1                                38.0                           5.0   
      2                                56.0                           8.0   
      3                                53.0                           8.0   
      4                                43.0                           6.0  


 num_days: 1
X_tmp:
    promo_14_2017  promo_60_2017  promo_140_2017  promo_3_2017_aft  \
0              0              0               0                 0   
1              0              0               0                 0   
2              0              0               0                 0   
3              0              0               0                 0   
4              0              2              34                 0   

   promo_7_2017_aft  promo_14_2017_aft  has_promo_mean_3  \
0                 0                  0               NaN   
1                 0                  0               NaN   
2                 0                  0               NaN   
3                 0                  0               NaN   
4                 0                  0               NaN   

   has_promo_mean_3_decay  no_promo_mean_3  no_promo_mean_3_decay  ...  \
0                     0.0         0.000000               0.000000  ...   
1                     0.0         1.245890            

X_tmp3 before reset_index:
                  store_class_promo_14_2017  store_class_promo_60_2017  \
class store_nbr                                                         
1002  1                               23.0                       30.0   
      2                               35.0                       49.0   
      3                               37.0                       51.0   
      4                               29.0                       35.0   
      5                               10.0                       13.0   

                 store_class_promo_140_2017  store_class_promo_3_2017_aft  \
class store_nbr                                                             
1002  1                                48.0                           5.0   
      2                                74.0                           9.0   
      3                                71.0                           7.0   
      4                                57.0                           7.0  


 num_days: 2
X_tmp:
    promo_14_2017  promo_60_2017  promo_140_2017  promo_3_2017_aft  \
0              0              0               0                 0   
1              0              0               0                 0   
2              0              0               0                 0   
3              0              0               0                 0   
4              0              0              31                 0   

   promo_7_2017_aft  promo_14_2017_aft  has_promo_mean_3  \
0                 0                  0               NaN   
1                 0                  0               NaN   
2                 0                  0               NaN   
3                 0                  0               NaN   
4                 0                  0               NaN   

   has_promo_mean_3_decay  no_promo_mean_3  no_promo_mean_3_decay  ...  \
0                     0.0         0.231049               0.561449  ...   
1                     0.0         0.462098            

...

X_tmp3 before reset_index:
                  store_class_promo_14_2017  store_class_promo_60_2017  \
class store_nbr                                                         
1002  1                               17.0                       40.0   
      2                               26.0                       63.0   
      3                               24.0                       63.0   
      4                               22.0                       52.0   
      5                               15.0                       26.0   

                 store_class_promo_140_2017  store_class_promo_3_2017_aft  \
class store_nbr                                                             
1002  1                                65.0                           4.0   
      2                               100.0                           5.0   
      3                                95.0                           5.0   
      4                                79.0                           4.0  


 num_days: 4
X_tmp:
    promo_14_2017  promo_60_2017  promo_140_2017  promo_3_2017_aft  \
0              0              0               0                 0   
1              0              0               0                 0   
2              0              0               0                 0   
3              0              0               0                 0   
4              0              0              30                 0   

   promo_7_2017_aft  promo_14_2017_aft  has_promo_mean_3  \
0                 0                  0               NaN   
1                 0                  0               NaN   
2                 0                  0               NaN   
3                 0                  0               NaN   
4                 0                  0               NaN   

   has_promo_mean_3_decay  no_promo_mean_3  no_promo_mean_3_decay  ...  \
0                     0.0         0.000000               0.000000  ...   
1                     0.0         0.462098            

X_tmp3 before reset_index:
                  store_class_promo_14_2017  store_class_promo_60_2017  \
class store_nbr                                                         
1002  1                               13.0                       48.0   
      2                               20.0                       73.0   
      3                               23.0                       74.0   
      4                               18.0                       63.0   
      5                               10.0                       31.0   

                 store_class_promo_140_2017  store_class_promo_3_2017_aft  \
class store_nbr                                                             
1002  1                                73.0                           3.0   
      2                               112.0                           7.0   
      3                               108.0                           9.0   
      4                                91.0                           8.0  


 num_days: 5
X_tmp:
    promo_14_2017  promo_60_2017  promo_140_2017  promo_3_2017_aft  \
0              0              0               0                 0   
1              0              0               0                 0   
2              0              0               0                 0   
3              0              0               0                 0   
4              0              0              30                 0   

   promo_7_2017_aft  promo_14_2017_aft  has_promo_mean_3  \
0                 0                  0               NaN   
1                 0                  0               NaN   
2                 0                  0               NaN   
3                 0                  0               NaN   
4                 0                  0               NaN   

   has_promo_mean_3_decay  no_promo_mean_3  no_promo_mean_3_decay  ...  \
0                     0.0         0.000000               0.000000  ...   
1                     0.0         0.693147            

X_tmp3 before reset_index:
                  store_class_promo_14_2017  store_class_promo_60_2017  \
class store_nbr                                                         
1002  1                               17.0                       57.0   
      2                               28.0                       89.0   
      3                               33.0                       94.0   
      4                               32.0                       83.0   
      5                               20.0                       45.0   

                 store_class_promo_140_2017  store_class_promo_3_2017_aft  \
class store_nbr                                                             
1002  1                                82.0                           2.0   
      2                               128.0                          10.0   
      3                               128.0                           9.0   
      4                               111.0                          11.0  

In [48]:
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

In [49]:
X_train.head(5)

,promo_14_2017,promo_60_2017,promo_140_2017,promo_3_2017_aft,promo_7_2017_aft,promo_14_2017_aft,has_promo_mean_3,has_promo_mean_3_decay,no_promo_mean_3,no_promo_mean_3_decay,...,store_class_promo_15,item_nbr,family,class,perishable,store_nbr,city,state,type,cluster
0,0,0,0,0,0,0,NaN,0.0,0.000000,0.000000,...,3.0,96995,12,1093,0,1,18,12,3,13
1,0,0,0,0,0,0,NaN,0.0,0.597253,1.681898,...,3.0,99197,12,1067,0,1,18,12,3,13
2,0,0,0,0,0,0,NaN,0.0,0.828302,2.346277,...,3.0,103520,12,1028,0,1,18,12,3,13
3,0,0,0,0,0,0,NaN,0.0,1.059351,2.830688,...,2.0,103665,5,2712,1,1,18,12,3,13
4,0,8,34,0,0,0,NaN,0.0,1.329661,3.637564,...,1.0,105574,12,1045,0,1,18,12,3,13


In [52]:
del X_l, y_l
X_val, y_val = prepare_dataset(df_2017, promo_2017, date(2017, 7, 26))

X_val2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 7, 26), is_train=False, name_prefix='item')
X_val2.index = df_2017_item.index
X_val2 = X_val2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

X_val3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, date(2017, 7, 26), is_train=False, name_prefix='store_class')
X_val3.index = df_2017_store_class.index
X_val3 = X_val3.reindex(df_2017_store_class.index).reset_index(drop=True)

X_val = pd.concat([X_val, X_val2, X_val3, items.reset_index(), stores.reset_index()], axis=1)

X_test = prepare_dataset(df_2017, promo_2017, date(2017, 8, 16), is_train=False)

X_test2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 8, 16), is_train=False, name_prefix='item')
X_test2.index = df_2017_item.index
X_test2 = X_test2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

X_test3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, date(2017, 8, 16), is_train=False, name_prefix='store_class')
X_test3.index = df_2017_store_class.index
X_test3 = X_test3.reindex(df_2017_store_class.index).reset_index(drop=True)

X_test = pd.concat([X_test, X_test2, X_test3, items.reset_index(), stores.reset_index()], axis=1)

del X_test2, X_val2, df_2017_item, promo_2017_item, df_2017_store_class, df_2017_promo_store_class, df_2017_store_class_index
gc.collect()

76

In [53]:
print("Training and predicting models...")
params = {
    'num_leaves': 80,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 16
}

Training and predicting models...


In [54]:
MAX_ROUNDS = 5000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * num_days) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

Step 1


C:\Users\edward chen\Anaconda3\envs\python37\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


MemoryError: Unable to allocate 3.65 GiB for an array with shape (488, 1005090) and data type float64

In [ ]:
print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

weight = items["perishable"] * 0.25 + 1
err = (y_val - np.array(val_pred).transpose())**2
err = err.sum(axis=1) * weight
err = np.sqrt(err.sum() / weight.sum() / 16)
print('nwrmsle = {}'.format(err))

y_val = np.array(val_pred).transpose()
df_preds = pd.DataFrame(
    y_val, index=df_2017.index,
    columns=pd.date_range("2017-07-26", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)
df_preds["unit_sales"] = np.clip(np.expm1(df_preds["unit_sales"]), 0, 1000)
df_preds.reset_index().to_csv('lgb_cv.csv', index=False)

In [ ]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb_sub.csv', float_format='%.4f', index=None)